# 02 - Single Qubit Experiments and Calibration

This notebook demonstrates single qubit experiments and calibration procedures in LeeQ.

## Learning Objectives
- Perform Rabi oscillation experiments
- Understand qubit calibration workflows
- Learn measurement primitives
- Practice with coherence time measurements

## Prerequisites
- Complete [01_basics.ipynb](01_basics.ipynb)
- Understand quantum single-qubit gates

## Setup and Configuration

Let's set up our simulation environment using the patterns from the previous tutorial, but with a focus on single qubit calibration and characterization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Core LeeQ imports
from leeq.chronicle import Chronicle, log_and_record
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

# Import calibration and characterization experiments
from leeq.experiments.builtin.basic.calibrations import (
    RabiAmplitudeCalibration, 
    MeasurementStatistics
)
from leeq.experiments.builtin.basic.characterizations import (
    T1Measurement,
    T2EchoMeasurement,
    T2RamseyMeasurement
)

print("Single qubit experiment modules imported successfully!")
print("\nImported experiment types:")
print("  - RabiAmplitudeCalibration: Find optimal pulse amplitudes")
print("  - MeasurementStatistics: Basic qubit state measurement")
print("  - T1Measurement: Relaxation time characterization")
print("  - T2RamseyMeasurement: Dephasing time measurement")
print("  - T2EchoMeasurement: Spin echo for removing low-frequency noise")

In [ ]:
# Start Chronicle logging for this tutorial session
import os
from datetime import datetime

# Clear any existing Chronicle instance and start fresh
try:
    Chronicle().stop_log()
except:
    pass

Chronicle().start_log()
log_dir = Chronicle().get_log_dir()

log_and_record("single_qubit_tutorial_start", {
    "notebook": "02_single_qubit",
    "focus": "calibration_and_characterization",
    "timestamp": datetime.now().isoformat()
})

print(f"Chronicle logging started: {log_dir}")
print(f"Absolute path: {os.path.abspath(log_dir)}")
print(f"Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Creating a Realistic Virtual Qubit

For this tutorial, we'll create a virtual transmon qubit with realistic parameters based on current superconducting quantum devices. We'll also set up the simulation environment using the same patterns from the existing `simulated_setup.py`.

In [ ]:
# Create a virtual transmon with realistic parameters
virtual_transmon = VirtualTransmon(
    name="CalibrationQubit",
    qubit_frequency=5040.4,     # MHz, typical transmon frequency
    anharmonicity=-198,         # MHz, negative anharmonicity
    t1=70,                      # μs, relaxation time
    t2=35,                      # μs, dephasing time  
    readout_frequency=9645.4,   # MHz, readout resonator frequency
    quiescent_state_distribution=np.asarray([0.85, 0.10, 0.04, 0.01])  # Thermal population
)

print(f"Virtual transmon created: {virtual_transmon.name}")
print(f"Qubit parameters:")
print(f"  Frequency: {virtual_transmon.qubit_frequency} MHz")
print(f"  Anharmonicity: {virtual_transmon.anharmonicity} MHz")
print(f"  T1: {virtual_transmon.t1} μs")
print(f"  T2: {virtual_transmon.t2} μs")
print(f"  Thermal distribution: {virtual_transmon.quiescent_state_distribution}")

# Log the qubit configuration
log_and_record("qubit_configuration", {
    "name": virtual_transmon.name,
    "frequency_mhz": virtual_transmon.qubit_frequency,
    "anharmonicity_mhz": virtual_transmon.anharmonicity,
    "t1_us": virtual_transmon.t1,
    "t2_us": virtual_transmon.t2,
    "thermal_populations": virtual_transmon.quiescent_state_distribution.tolist()
})

In [ ]:
# Set up the simulation environment
manager = ExperimentManager()
manager.clear_setups()

# Create high-level simulation setup
setup = HighLevelSimulationSetup(
    name='SingleQubitCalibrationSetup',
    virtual_qubits={1: virtual_transmon}  # Assign to logical channel 1
)

# Register the setup
manager.register_setup(setup)

print(f"Simulation setup registered: {setup.name}")
print(f"Available qubits: {list(setup.virtual_qubits.keys())}")

# Define the qubit configuration following existing patterns
qubit_config = {
    'hrid': 'Q1',
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': virtual_transmon.qubit_frequency,
            'channel': 1,
            'shape': 'blackman_drag',
            'amp': 0.5,  # Initial amplitude estimate
            'phase': 0.,
            'width': 0.05,  # 50 ns pulse width
            'alpha': 500,   # DRAG parameter
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': virtual_transmon.readout_frequency,
            'channel': 1,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.,
            'width': 1,     # 1 μs measurement
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

log_and_record("qubit_configuration_details", qubit_config)
print("Qubit control configuration defined")

## Rabi Oscillation Experiments

Rabi oscillations are fundamental to qubit calibration. By sweeping the drive amplitude, we can:
- Find the π-pulse amplitude (flips |0⟩ → |1⟩)
- Measure the Rabi frequency
- Assess drive coherence

### Theory
For a resonant drive, the Rabi frequency is:
$$\Omega_R = \frac{\mu \cdot E}{\hbar}$$

where μ is the dipole moment and E is the electric field amplitude.

In [ ]:
# Create a Rabi amplitude calibration experiment
rabi_experiment = RabiAmplitudeCalibration(
    name="RabiAmplitudeCalibration",
    qubit=1,
    drive_frequency=virtual_transmon.qubit_frequency,
    amplitude_start=0.0,
    amplitude_stop=1.0,
    amplitude_points=51,
    pulse_width=0.05,  # 50 ns
    repeated_measurement_count=1000
)

print(f"Rabi experiment configured:")
print(f"  Amplitude range: {rabi_experiment.amplitude_start} to {rabi_experiment.amplitude_stop}")
print(f"  Number of points: {rabi_experiment.amplitude_points}")
print(f"  Pulse width: {rabi_experiment.pulse_width} μs")
print(f"  Measurements per point: {rabi_experiment.repeated_measurement_count}")

# Log experiment parameters
log_and_record("rabi_experiment_setup", {
    "experiment_type": "RabiAmplitudeCalibration",
    "qubit": rabi_experiment.qubit,
    "drive_frequency": rabi_experiment.drive_frequency,
    "amplitude_range": [rabi_experiment.amplitude_start, rabi_experiment.amplitude_stop],
    "points": rabi_experiment.amplitude_points,
    "pulse_width_us": rabi_experiment.pulse_width
})

In [ ]:
# Run the Rabi experiment
print("Running Rabi amplitude calibration...")
rabi_results = rabi_experiment.run()

# Extract data for analysis
amplitudes = rabi_results['sweep_values']
excited_probabilities = rabi_results['measurement_probabilities']

print(f"Rabi experiment completed with {len(amplitudes)} data points")
print(f"Amplitude range: {amplitudes[0]:.3f} to {amplitudes[-1]:.3f}")
print(f"Excited state probability range: {min(excited_probabilities):.3f} to {max(excited_probabilities):.3f}")

# Find π-pulse amplitude (first maximum)
max_idx = np.argmax(excited_probabilities)
pi_pulse_amplitude = amplitudes[max_idx]
max_excited_prob = excited_probabilities[max_idx]

print(f"\nPi-pulse amplitude: {pi_pulse_amplitude:.3f}")
print(f"Maximum excited probability: {max_excited_prob:.3f}")

# Log results
log_and_record("rabi_results", {
    "pi_pulse_amplitude": pi_pulse_amplitude,
    "max_excited_probability": max_excited_prob,
    "amplitudes": amplitudes.tolist(),
    "probabilities": excited_probabilities.tolist()
})

In [ ]:
# Visualize Rabi oscillations
plt.figure(figsize=(12, 8))

# Main Rabi plot
plt.subplot(2, 2, 1)
plt.plot(amplitudes, excited_probabilities, 'bo-', markersize=4, linewidth=2)
plt.axvline(pi_pulse_amplitude, color='red', linestyle='--', alpha=0.7, 
            label=f'π-pulse: {pi_pulse_amplitude:.3f}')
plt.xlabel('Drive Amplitude')
plt.ylabel('Excited State Probability')
plt.title('Rabi Oscillations')
plt.grid(alpha=0.3)
plt.legend()

# Fit Rabi frequency (simplified damped cosine)
def rabi_model(amp, a, b, freq, phase, offset):
    return offset + a * np.exp(-b * amp) * np.cos(2 * np.pi * freq * amp + phase)

try:
    # Initial parameter guess
    p0 = [0.4, 0.5, 2.0, 0, 0.15]  # amplitude, decay, frequency, phase, offset
    popt, pcov = curve_fit(rabi_model, amplitudes, excited_probabilities, p0=p0)
    
    # Generate smooth fit curve
    amp_fit = np.linspace(amplitudes[0], amplitudes[-1], 200)
    prob_fit = rabi_model(amp_fit, *popt)
    
    plt.subplot(2, 2, 2)
    plt.plot(amplitudes, excited_probabilities, 'bo', markersize=4, label='Data')
    plt.plot(amp_fit, prob_fit, 'r-', linewidth=2, label='Fit')
    plt.xlabel('Drive Amplitude')
    plt.ylabel('Excited State Probability')
    plt.title('Rabi Fit')
    plt.grid(alpha=0.3)
    plt.legend()
    
    rabi_frequency = popt[2]
    print(f"Fitted Rabi frequency: {rabi_frequency:.3f} (amplitude units)^-1")
    
    log_and_record("rabi_fit", {
        "fit_parameters": popt.tolist(),
        "rabi_frequency": rabi_frequency
    })
    
except Exception as e:
    print(f"Fit failed: {e}")
    plt.subplot(2, 2, 2)
    plt.text(0.5, 0.5, 'Fit Failed', ha='center', va='center', transform=plt.gca().transAxes)

# Show pulse amplitude vs Rabi angle
plt.subplot(2, 2, 3)
# Estimate Rabi angles based on amplitude
rabi_angles = amplitudes * (np.pi / pi_pulse_amplitude)  # Normalize to π
plt.plot(rabi_angles, excited_probabilities, 'go-', markersize=4, linewidth=2)
plt.axvline(np.pi, color='red', linestyle='--', alpha=0.7, label='π pulse')
plt.axvline(np.pi/2, color='orange', linestyle='--', alpha=0.7, label='π/2 pulse')
plt.xlabel('Rabi Angle (radians)')
plt.ylabel('Excited State Probability')
plt.title('Rabi Angle vs Excited Probability')
plt.grid(alpha=0.3)
plt.legend()

# Summary statistics
plt.subplot(2, 2, 4)
stats_text = f"""Rabi Calibration Summary

π-pulse amplitude: {pi_pulse_amplitude:.3f}
π/2-pulse amplitude: {pi_pulse_amplitude/2:.3f}
Max excited prob: {max_excited_prob:.3f}
Contrast: {max_excited_prob - min(excited_probabilities):.3f}

Pulse width: {rabi_experiment.pulse_width} μs
Drive frequency: {rabi_experiment.drive_frequency} MHz
"""
plt.text(0.1, 0.9, stats_text, transform=plt.gca().transAxes, 
         verticalalignment='top', fontfamily='monospace')
plt.axis('off')

plt.tight_layout()
plt.show()

# Save calibrated amplitudes for later use
calibrated_amplitudes = {
    'pi_pulse': pi_pulse_amplitude,
    'pi_half_pulse': pi_pulse_amplitude / 2,
    'pi_quarter_pulse': pi_pulse_amplitude / 4
}

log_and_record("calibrated_amplitudes", calibrated_amplitudes)
print(f"\nCalibrated pulse amplitudes saved: {calibrated_amplitudes}")

## Coherence Time Measurements

Coherence times are critical parameters that determine the fidelity and duration of quantum operations. We'll measure:

### T1 (Relaxation Time)
- Time for excited state to decay to ground state
- Limited by energy dissipation to environment
- Measured by preparing |1⟩ and waiting variable delay before measurement

### T2 (Dephasing Time)  
- Time for coherent superposition to lose phase information
- Can be measured using Ramsey or spin echo sequences
- T2* (Ramsey) includes low-frequency noise, T2 (echo) removes it

In [ ]:
# T1 Measurement: Relaxation time
t1_experiment = T1Measurement(
    name="T1Measurement",
    qubit=1,
    delay_start=0.0,      # Start at 0 μs
    delay_stop=200.0,     # End at 200 μs (longer than expected T1)
    delay_points=41,      # 41 delay points
    pi_pulse_amplitude=calibrated_amplitudes['pi_pulse'],
    repeated_measurement_count=1000
)

print(f"T1 experiment configured:")
print(f"  Delay range: {t1_experiment.delay_start} to {t1_experiment.delay_stop} μs")
print(f"  Number of points: {t1_experiment.delay_points}")
print(f"  Pi-pulse amplitude: {t1_experiment.pi_pulse_amplitude:.3f}")
print(f"  Measurements per point: {t1_experiment.repeated_measurement_count}")

log_and_record("t1_experiment_setup", {
    "experiment_type": "T1Measurement",
    "qubit": t1_experiment.qubit,
    "delay_range_us": [t1_experiment.delay_start, t1_experiment.delay_stop],
    "points": t1_experiment.delay_points,
    "pi_pulse_amplitude": t1_experiment.pi_pulse_amplitude
})

In [ ]:
# Run T1 measurement
print("Running T1 measurement...")
t1_results = t1_experiment.run()

# Extract data
t1_delays = t1_results['sweep_values']
t1_probabilities = t1_results['measurement_probabilities']

print(f"T1 measurement completed with {len(t1_delays)} data points")
print(f"Delay range: {t1_delays[0]:.1f} to {t1_delays[-1]:.1f} μs")
print(f"Initial excited probability: {t1_probabilities[0]:.3f}")
print(f"Final excited probability: {t1_probabilities[-1]:.3f}")

# Fit exponential decay: P(t) = A * exp(-t/T1) + B
def exponential_decay(t, amplitude, t1, offset):
    return amplitude * np.exp(-t / t1) + offset

try:
    # Initial guess: amplitude, T1, offset
    p0 = [0.8, 70, 0.1]  # Start with expected T1 around 70 μs
    popt_t1, pcov_t1 = curve_fit(exponential_decay, t1_delays, t1_probabilities, p0=p0)
    
    fitted_t1 = popt_t1[1]
    amplitude = popt_t1[0]
    offset = popt_t1[2]
    
    print(f"\nT1 fit results:")
    print(f"  Fitted T1: {fitted_t1:.1f} μs")
    print(f"  Amplitude: {amplitude:.3f}")
    print(f"  Offset: {offset:.3f}")
    
    # Calculate confidence interval
    t1_error = np.sqrt(pcov_t1[1, 1])
    print(f"  T1 uncertainty: ±{t1_error:.1f} μs")
    
    log_and_record("t1_fit_results", {
        "fitted_t1_us": fitted_t1,
        "amplitude": amplitude,
        "offset": offset,
        "t1_error_us": t1_error,
        "expected_t1_us": virtual_transmon.t1
    })
    
    t1_fit_successful = True
    
except Exception as e:
    print(f"T1 fit failed: {e}")
    t1_fit_successful = False
    fitted_t1 = None

log_and_record("t1_measurement_results", {
    "delays_us": t1_delays.tolist(),
    "probabilities": t1_probabilities.tolist(),
    "fit_successful": t1_fit_successful
})

In [ ]:
# T2 Ramsey Measurement: Dephasing time with low-frequency noise
t2_ramsey_experiment = T2RamseyMeasurement(
    name="T2RamseyMeasurement",
    qubit=1,
    delay_start=0.0,
    delay_stop=100.0,     # Up to 100 μs
    delay_points=51,
    pi_half_pulse_amplitude=calibrated_amplitudes['pi_half_pulse'],
    detuning=0.1,         # 0.1 MHz detuning for oscillations
    repeated_measurement_count=1000
)

print(f"T2 Ramsey experiment configured:")
print(f"  Delay range: {t2_ramsey_experiment.delay_start} to {t2_ramsey_experiment.delay_stop} μs")
print(f"  Number of points: {t2_ramsey_experiment.delay_points}")
print(f"  π/2-pulse amplitude: {t2_ramsey_experiment.pi_half_pulse_amplitude:.3f}")
print(f"  Detuning: {t2_ramsey_experiment.detuning} MHz")

# Run T2 Ramsey measurement
print("\nRunning T2 Ramsey measurement...")
t2_ramsey_results = t2_ramsey_experiment.run()

# Extract data
t2_delays = t2_ramsey_results['sweep_values']
t2_probabilities = t2_ramsey_results['measurement_probabilities']

print(f"T2 Ramsey measurement completed with {len(t2_delays)} data points")

# Fit damped oscillation: P(t) = A * exp(-t/T2*) * cos(2π*f*t + φ) + B
def ramsey_model(t, amplitude, t2_star, frequency, phase, offset):
    return amplitude * np.exp(-t / t2_star) * np.cos(2 * np.pi * frequency * t + phase) + offset

try:
    # Initial guess
    p0 = [0.4, 30, t2_ramsey_experiment.detuning, 0, 0.5]
    popt_t2, pcov_t2 = curve_fit(ramsey_model, t2_delays, t2_probabilities, p0=p0)
    
    fitted_t2_star = popt_t2[1]
    fitted_frequency = popt_t2[2]
    
    print(f"\nT2* fit results:")
    print(f"  Fitted T2*: {fitted_t2_star:.1f} μs")
    print(f"  Fitted frequency: {fitted_frequency:.3f} MHz")
    print(f"  Expected frequency: {t2_ramsey_experiment.detuning:.3f} MHz")
    
    t2_ramsey_fit_successful = True
    
    log_and_record("t2_ramsey_fit_results", {
        "fitted_t2_star_us": fitted_t2_star,
        "fitted_frequency_mhz": fitted_frequency,
        "expected_t2_us": virtual_transmon.t2
    })
    
except Exception as e:
    print(f"T2 Ramsey fit failed: {e}")
    t2_ramsey_fit_successful = False
    fitted_t2_star = None

log_and_record("t2_ramsey_measurement_results", {
    "delays_us": t2_delays.tolist(),
    "probabilities": t2_probabilities.tolist(),
    "fit_successful": t2_ramsey_fit_successful
})

In [ ]:
# Visualize coherence time measurements
plt.figure(figsize=(15, 10))

# T1 measurement plot
plt.subplot(2, 3, 1)
plt.plot(t1_delays, t1_probabilities, 'bo', markersize=4, label='Data')
if t1_fit_successful:
    t_fit = np.linspace(t1_delays[0], t1_delays[-1], 200)
    p_fit = exponential_decay(t_fit, *popt_t1)
    plt.plot(t_fit, p_fit, 'r-', linewidth=2, label=f'Fit: T1={fitted_t1:.1f}μs')
    plt.axhline(popt_t1[2] + popt_t1[0]/np.e, color='gray', linestyle='--', alpha=0.7, label='1/e')
plt.xlabel('Delay (μs)')
plt.ylabel('Excited State Probability')
plt.title('T1 Relaxation Measurement')
plt.grid(alpha=0.3)
plt.legend()

# T2 Ramsey measurement plot
plt.subplot(2, 3, 2)
plt.plot(t2_delays, t2_probabilities, 'go', markersize=4, label='Data')
if t2_ramsey_fit_successful:
    t_fit = np.linspace(t2_delays[0], t2_delays[-1], 200)
    p_fit = ramsey_model(t_fit, *popt_t2)
    plt.plot(t_fit, p_fit, 'r-', linewidth=2, label=f'Fit: T2*={fitted_t2_star:.1f}μs')
    # Plot envelope
    envelope = popt_t2[0] * np.exp(-t_fit / popt_t2[1]) + popt_t2[4]
    plt.plot(t_fit, envelope, 'k--', alpha=0.5, label='Envelope')
    plt.plot(t_fit, 2*popt_t2[4] - envelope, 'k--', alpha=0.5)
plt.xlabel('Delay (μs)')
plt.ylabel('Excited State Probability')
plt.title('T2* Ramsey Measurement')
plt.grid(alpha=0.3)
plt.legend()

# Coherence time comparison
plt.subplot(2, 3, 3)
times_measured = []
times_expected = []
labels = []

if t1_fit_successful:
    times_measured.append(fitted_t1)
    times_expected.append(virtual_transmon.t1)
    labels.append('T1')

if t2_ramsey_fit_successful:
    times_measured.append(fitted_t2_star)
    times_expected.append(virtual_transmon.t2)
    labels.append('T2*')

if times_measured:
    x_pos = np.arange(len(labels))
    width = 0.35
    
    plt.bar(x_pos - width/2, times_expected, width, label='Expected', alpha=0.7, color='blue')
    plt.bar(x_pos + width/2, times_measured, width, label='Measured', alpha=0.7, color='red')
    
    plt.xlabel('Coherence Time Type')
    plt.ylabel('Time (μs)')
    plt.title('Expected vs Measured Coherence Times')
    plt.xticks(x_pos, labels)
    plt.legend()
    plt.grid(alpha=0.3)
else:
    plt.text(0.5, 0.5, 'No successful fits', ha='center', va='center', transform=plt.gca().transAxes)

# T1 decay rate analysis
plt.subplot(2, 3, 4)
if t1_fit_successful:
    # Show log-scale decay
    plt.semilogy(t1_delays, t1_probabilities - popt_t1[2], 'bo', markersize=4, label='Data')
    t_fit = np.linspace(t1_delays[0], t1_delays[-1], 200)
    p_fit_log = popt_t1[0] * np.exp(-t_fit / popt_t1[1])
    plt.semilogy(t_fit, p_fit_log, 'r-', linewidth=2, label='Exponential fit')
    plt.xlabel('Delay (μs)')
    plt.ylabel('Excess Probability (log scale)')
    plt.title('T1 Exponential Decay (Log Scale)')
    plt.grid(alpha=0.3)
    plt.legend()
else:
    plt.text(0.5, 0.5, 'T1 fit failed', ha='center', va='center', transform=plt.gca().transAxes)

# Ramsey frequency analysis
plt.subplot(2, 3, 5)
if t2_ramsey_fit_successful:
    # Show Fourier transform of oscillations
    from scipy.fft import fft, fftfreq
    
    # Remove DC component and apply window
    signal = t2_probabilities - np.mean(t2_probabilities)
    windowed_signal = signal * np.hanning(len(signal))
    
    fft_signal = np.abs(fft(windowed_signal))
    fft_freqs = fftfreq(len(signal), d=(t2_delays[1] - t2_delays[0]))
    
    # Plot positive frequencies only
    pos_freqs = fft_freqs[fft_freqs > 0]
    pos_fft = fft_signal[fft_freqs > 0]
    
    plt.plot(pos_freqs, pos_fft, 'g-', linewidth=2)
    plt.axvline(fitted_frequency, color='red', linestyle='--', alpha=0.7, 
                label=f'Fitted: {fitted_frequency:.3f} MHz')
    plt.axvline(t2_ramsey_experiment.detuning, color='blue', linestyle='--', alpha=0.7,
                label=f'Expected: {t2_ramsey_experiment.detuning:.3f} MHz')
    plt.xlabel('Frequency (MHz)')
    plt.ylabel('FFT Amplitude')
    plt.title('Ramsey Frequency Spectrum')
    plt.grid(alpha=0.3)
    plt.legend()
    plt.xlim(0, 0.5)  # Focus on low frequencies
else:
    plt.text(0.5, 0.5, 'T2* fit failed', ha='center', va='center', transform=plt.gca().transAxes)

# Summary statistics
plt.subplot(2, 3, 6)
summary_text = f"""Coherence Measurement Summary

T1 Relaxation:
  Expected: {virtual_transmon.t1:.1f} μs
  Measured: {fitted_t1:.1f} μs" if t1_fit_successful else "Failed

T2* Dephasing (Ramsey):
  Expected: {virtual_transmon.t2:.1f} μs
  Measured: {fitted_t2_star:.1f} μs" if t2_ramsey_fit_successful else "Failed

Calibrated Pulses:
  π-pulse: {calibrated_amplitudes['pi_pulse']:.3f}
  π/2-pulse: {calibrated_amplitudes['pi_half_pulse']:.3f}

Qubit Frequency: {virtual_transmon.qubit_frequency} MHz
Anharmonicity: {virtual_transmon.anharmonicity} MHz
"""

plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes,
         verticalalignment='top', fontfamily='monospace', fontsize=9)
plt.axis('off')

plt.tight_layout()
plt.show()

# Final summary for Chronicle
coherence_summary = {
    "t1_expected_us": virtual_transmon.t1,
    "t1_measured_us": fitted_t1 if t1_fit_successful else None,
    "t2_expected_us": virtual_transmon.t2,
    "t2_star_measured_us": fitted_t2_star if t2_ramsey_fit_successful else None,
    "measurement_quality": {
        "t1_fit_successful": t1_fit_successful,
        "t2_ramsey_fit_successful": t2_ramsey_fit_successful
    }
}

log_and_record("coherence_measurement_summary", coherence_summary)
print("\nCoherence measurements completed and logged to Chronicle")

## Single Qubit Calibration Workflow

Now let's put together a complete single qubit calibration workflow that demonstrates best practices for quantum device characterization.

In [ ]:
# Define a complete single-qubit calibration workflow
def single_qubit_calibration_workflow(qubit_id, virtual_transmon, setup):
    """
    Complete single qubit calibration workflow.
    
    Returns calibrated parameters and characterization results.
    """
    workflow_results = {}
    
    print(f"Starting calibration workflow for qubit {qubit_id}")
    log_and_record("calibration_workflow_start", {
        "qubit_id": qubit_id,
        "qubit_name": virtual_transmon.name
    })
    
    # Step 1: Basic measurement statistics
    print("Step 1: Basic measurement statistics...")
    basic_measurement = MeasurementStatistics(
        name=f"BasicMeasurement_Q{qubit_id}",
        qubit=qubit_id,
        repeated_measurement_count=1000
    )
    basic_results = basic_measurement.run()
    workflow_results['basic_measurement'] = basic_results
    
    print(f"  Ground state fidelity: {basic_results['statistics']['ground_state_probability']:.3f}")
    
    # Step 2: Rabi calibration
    print("Step 2: Rabi amplitude calibration...")
    rabi_cal = RabiAmplitudeCalibration(
        name=f"RabiCalibration_Q{qubit_id}",
        qubit=qubit_id,
        drive_frequency=virtual_transmon.qubit_frequency,
        amplitude_start=0.0,
        amplitude_stop=1.0,
        amplitude_points=31,  # Fewer points for faster calibration
        pulse_width=0.05,
        repeated_measurement_count=500
    )
    rabi_results = rabi_cal.run()
    
    # Find optimal pi-pulse amplitude
    amplitudes = rabi_results['sweep_values']
    probabilities = rabi_results['measurement_probabilities']
    pi_amp = amplitudes[np.argmax(probabilities)]
    
    workflow_results['rabi_calibration'] = {
        'pi_amplitude': pi_amp,
        'max_excited_prob': np.max(probabilities),
        'amplitudes': amplitudes,
        'probabilities': probabilities
    }
    
    print(f"  Calibrated π-pulse amplitude: {pi_amp:.3f}")
    
    # Step 3: T1 characterization
    print("Step 3: T1 relaxation measurement...")
    t1_char = T1Measurement(
        name=f"T1Characterization_Q{qubit_id}",
        qubit=qubit_id,
        delay_start=0.0,
        delay_stop=150.0,
        delay_points=21,  # Fewer points for faster measurement
        pi_pulse_amplitude=pi_amp,
        repeated_measurement_count=500
    )
    t1_results = t1_char.run()
    
    # Fit T1
    t1_delays = t1_results['sweep_values']
    t1_probs = t1_results['measurement_probabilities']
    
    try:
        popt_t1, _ = curve_fit(exponential_decay, t1_delays, t1_probs, 
                              p0=[0.8, 70, 0.1])
        fitted_t1 = popt_t1[1]
        t1_fit_success = True
    except:
        fitted_t1 = None
        t1_fit_success = False
    
    workflow_results['t1_characterization'] = {
        'fitted_t1': fitted_t1,
        'fit_successful': t1_fit_success,
        'delays': t1_delays,
        'probabilities': t1_probs
    }
    
    if t1_fit_success:
        print(f"  Measured T1: {fitted_t1:.1f} μs")
    else:
        print("  T1 fit failed")
    
    # Step 4: T2* Ramsey characterization
    print("Step 4: T2* Ramsey dephasing measurement...")
    t2_char = T2RamseyMeasurement(
        name=f"T2RamseyCharacterization_Q{qubit_id}",
        qubit=qubit_id,
        delay_start=0.0,
        delay_stop=80.0,
        delay_points=21,
        pi_half_pulse_amplitude=pi_amp/2,
        detuning=0.1,
        repeated_measurement_count=500
    )
    t2_results = t2_char.run()
    
    # Fit T2*
    t2_delays = t2_results['sweep_values']
    t2_probs = t2_results['measurement_probabilities']
    
    try:
        popt_t2, _ = curve_fit(ramsey_model, t2_delays, t2_probs, 
                              p0=[0.4, 30, 0.1, 0, 0.5])
        fitted_t2_star = popt_t2[1]
        t2_fit_success = True
    except:
        fitted_t2_star = None
        t2_fit_success = False
    
    workflow_results['t2_characterization'] = {
        'fitted_t2_star': fitted_t2_star,
        'fit_successful': t2_fit_success,
        'delays': t2_delays,
        'probabilities': t2_probs
    }
    
    if t2_fit_success:
        print(f"  Measured T2*: {fitted_t2_star:.1f} μs")
    else:
        print("  T2* fit failed")
    
    # Step 5: Generate calibration summary
    calibration_summary = {
        'qubit_id': qubit_id,
        'qubit_name': virtual_transmon.name,
        'calibrated_parameters': {
            'pi_pulse_amplitude': pi_amp,
            'pi_half_pulse_amplitude': pi_amp/2,
            'drive_frequency_mhz': virtual_transmon.qubit_frequency,
            'readout_frequency_mhz': virtual_transmon.readout_frequency
        },
        'characterized_parameters': {
            't1_us': fitted_t1 if t1_fit_success else None,
            't2_star_us': fitted_t2_star if t2_fit_success else None,
            'ground_state_fidelity': basic_results['statistics']['ground_state_probability'],
            'max_excited_fidelity': workflow_results['rabi_calibration']['max_excited_prob']
        },
        'expected_parameters': {
            't1_us': virtual_transmon.t1,
            't2_us': virtual_transmon.t2,
            'anharmonicity_mhz': virtual_transmon.anharmonicity
        }
    }
    
    workflow_results['summary'] = calibration_summary
    
    log_and_record("calibration_workflow_complete", calibration_summary)
    print(f"Calibration workflow completed for qubit {qubit_id}")
    
    return workflow_results

# Run the complete calibration workflow
print("Running complete single qubit calibration workflow...")
calibration_results = single_qubit_calibration_workflow(1, virtual_transmon, setup)

print("\n" + "="*60)
print("CALIBRATION WORKFLOW SUMMARY")
print("="*60)
summary = calibration_results['summary']
print(f"Qubit: {summary['qubit_name']} (Channel {summary['qubit_id']})")
print(f"\nCalibrated Parameters:")
for param, value in summary['calibrated_parameters'].items():
    print(f"  {param}: {value:.3f}")
print(f"\nCharacterized Parameters:")
for param, value in summary['characterized_parameters'].items():
    if value is not None:
        print(f"  {param}: {value:.3f}")
    else:
        print(f"  {param}: Measurement failed")
print("="*60)

## Key Takeaways

From this single qubit tutorial, you've learned:

1. **Rabi Calibration**: How to find optimal pulse amplitudes for precise qubit control
2. **Coherence Measurements**: T1 and T2* characterization using exponential fits
3. **Calibration Workflows**: Systematic approach to qubit characterization
4. **Data Analysis**: Fitting experimental data to extract meaningful parameters
5. **Chronicle Integration**: Automatic logging of all experimental data and results

### Understanding the Results

- **Rabi Oscillations**: Show the relationship between drive amplitude and state flip probability
- **T1 Decay**: Measures energy relaxation from |1⟩ to |0⟩
- **T2* Ramsey**: Measures dephasing including low-frequency noise
- **Calibration Quality**: Compare measured vs expected values to assess simulation accuracy

### Next Steps

- **Practice**: Try different qubit parameters and observe how they affect measurements
- **Explore**: Examine the Chronicle logs to see the complete experimental record
- **Advanced**: Continue to [03_multi_qubit.ipynb](03_multi_qubit.ipynb) for two-qubit experiments

### Best Practices

1. **Always calibrate before characterization**: Get accurate pulse amplitudes first
2. **Use appropriate measurement statistics**: More averages for critical calibrations
3. **Validate fits**: Check that fitted parameters make physical sense
4. **Log everything**: Use Chronicle to maintain complete experimental records
5. **Systematic approach**: Follow consistent workflows for reproducible results

In [ ]:
# Clean up and finalize logging
log_and_record("single_qubit_tutorial_complete", {
    "notebook": "02_single_qubit",
    "experiments_completed": [
        "RabiAmplitudeCalibration",
        "T1Measurement", 
        "T2RamseyMeasurement",
        "SingleQubitCalibrationWorkflow"
    ],
    "key_concepts": [
        "rabi_oscillations",
        "coherence_times",
        "calibration_workflows",
        "experimental_fitting",
        "qubit_characterization"
    ],
    "final_calibration": calibration_results['summary']
})

print("\n" + "="*50)
print("Tutorial 02 - Single Qubit Experiments completed successfully!")
print("Next: 03_multi_qubit.ipynb")
print("="*50)